In [1]:
# References
# 1. https://www.kaggle.com/ammarnassanalhajali/cnn-with-keras-stater
# 2. https://www.kaggle.com/vsedelnik/happywhale-dataset-image-normalization

In [2]:
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow
from tqdm.autonotebook import tqdm

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import keras.backend as K
from keras.models import Sequential
from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Model

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [3]:
train = pd.read_csv('../input/happy-whale-and-dolphin/train.csv')
print(f"Shape of TrainSet: {train.shape}\n")
train.head()

In [10]:
# Functions

def Loading_Images(data, m, dataset):
    print("Loading Images")
    X_train = np.zeros((m, 32, 32, 3))    # 높이*행*열*채널
    count = 0
    for fig in tqdm(data['image']):
        img = image.load_img("../input/happy-whale-and-dolphin/"+dataset+'/'+fig,
                             target_size=(32, 32, 3))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        X_train[count] = x
        count += 1
    return X_train
# X_train 생성(zero matrix) > Image load > Convert to Array > Normalization > X_train
# preprocess_input: 이미지를 -255에서 255사이로 Normalization하고 RGB값을 BGR값으로 변경합니다. BGR을RGB로 변경하던가
 
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    onehot_encoder = OneHotEncoder(sparse=False)    # default: True(matrix 반환)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    y = onehot_encoded
    return y, label_encoder
# label encoding: 순서대로 숫자 할당
# onehot encoding: 더미화

In [5]:
X = Loading_Images(train, train.shape[0], 'train_images')
X /= 255

In [11]:
y, label_encoder = prepare_labels(train['individual_id'])

In [12]:
model = Sequential()

model.add(Conv2D(32,
                (6,6),
                strides=(1,1),
                input_shape-(32, 32, 3)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))

model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,
                (3, 3),
                strides=(1,1)))
model.add(Activation('relu'))
model.add(AveragePooling2D((3,3)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.85))

model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

In [13]:
history = model.fit(X, y, epochs=150, batch_size=128, verbose=1)
model.save('/last.h5')

In [14]:
# Evaluation

plt.figure(figsize=(15, 5))
plt.plot(history.history['accuracy'])
plt.title('Model Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

In [15]:
plt.figure(figsize=(15, 5))
plt.plot(history.history['loss'])
plt.title("Model Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [16]:
# Inference

test = os.listdir("../input/happy-whale-and-dolphin/test_images")
print(len(test))

In [17]:
col = ['image']
test_df = pd.DataFrame(test, columns=col)
test_df['predictions'] = ''

In [18]:
batch_size=5000
batch_start=0
batch_end = batch_size
L = len(test_df)

while batch_start < L:
    limit = min(batch_end, L)
    test_df_batch = test_df.iloc[batch_start:limit]
    print(type(test_df_batch))
    X = Loading_Images(test_df_batch, test_df_batch.shape[0], 'test_images')
    X /= 255
    predictions = model.predicct(np.array(X), verbose=1)
    
    for i, pred in enumerate(predictions):
        p = pred.argsort()[-5:][::-1]    # 5번째부터 역순 정렬
        idx = -1
        s, s1, s2 = '', '', ''
        for x in p:
            idx = idx+1
            if pred[x] > 0.6:
                s1 = s1+' '+label_encoder.inverse_transform(p)[idx]
            else:
                s2 = s2+' '+label_encoder.inverse_transform(p)[idx]
        print(s1, s2)
        s = s1+' new_individual'+s2
        s = s.strip(' ')
        test_df.loc[batch_start+i, 'predictions'] = s
    batch_start += batch_size
    batch_end += batch_size
    

In [ ]:
test_df.to_csv("submission.csv", index=False)
test_df.head()

# Image Normalization

In [19]:
import os
import random
from multiprocessing import Pool
import cv2
import albumentations as A
INPUT_PATH = '../input/happy-whale-and-dolphin'

In [20]:
train_files = os.listdir(os.path.join(INPUT_PATH, 'train_images'))
test_files = os.listdir(os.path.join(INPUT_PATH, 'test_images'))
all_files = [os.path.join(INPUT_PATH, 'train_images', f) for f in train_files]+ \
            [os.path.join(INPUT_PATH, 'test_images', f) for f in test_files]

print(f"Train files:{len(train_files)}, test_files: {len(test_files)}, all_files: {len(all_files)}")

show_images = random.sample(all_files, 5)

In [21]:
def show_orig_norm_images(img_files, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)):
    nrows, ncols = len(img_files), 2
    fig, ax = plt.subplots(nrows, ncols, figsize=(20, 31))
    for i in range(len(img_files)):
        img = cv2.imread(img_files[i])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        norm_img = A.Normalize(mean=mean, std=std)(image=img)['image']
        print(img.shape)
        print(norm_img[0])
        ax[i, 0].grid(False)
        ax[i, 0].axis('off')
        ax[i, 0].title.set_text(f'{os.path.basename(img_files[i])}: original')
        ax[i, 0].imshow(img)
        
        ax[i, 1].grid(False)
        ax[i, 1].axis('off')
        ax[i, 1].title.set_text(f'{os.path.basename(img_files[i])}: normalized')
        ax[i, 1].imshow(norm_img)
        
        plt.tight_layout()
        plt.show()
        
show_orig_norm_images(show_images)

In [ ]:
# 각 이미지별, 각 컬러 채널별 픽셀 평균, 분산 계산
# 이후 평균, 분산은 데이터셋의 모든 이미지들에 대해 평균

In [22]:
np.set_printoptions(precisions=3)

all_files = all_files[:1000]

def process_file(fp):
    img = cv2.imread(fp)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255
    return np.mean(img, axis=(0, 1)), np.std(img, axis=(0, 1))

mean, std = np.zeros(3), np.zeros(3)
n, done = len(all_files), 0

with Pool(os.cpu_count()) as p:
    pbar = tqdm(p.imap(process_file, all_files), total=n)
    for m, s in pbar:
        done += 1
        mean += m
        std += s
        pbar.set_description(f"{mean/done} {std/done}")
        
mean, std = mean/n, std/n

print(f"Calculated mean: {mean}")
print(f"Calculated std: {std}")

Normalize는 입력 받은 이미지 값의 범위를 (0, 255) → (-1, 1) 범위로 줄여주는 역할

이와 같이 하는 이유는 입력 값의 범위를 줄여줌으로써 학습이 빨리 수렴되게 하고 특정 입력값이 커짐으로써 특정 weight값이 커지는 문제를 개선할 수 있음

